In [61]:
import json
import glob

from transformers import LongformerTokenizerFast
from collections import defaultdict

In [60]:
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-large-4096', add_prefix_space=True)

data_files = glob.glob("/home/shtoshni/Research/litbank_coref/data/character_identification/orig/character*.json")

In [82]:
mention_count = 0
uniq_entities = set()

from collections import Counter
cluster_lens = []
for data_file in data_files:
    data = json.load(open(data_file))
    for episode in data['episodes']:
        
        for scene in episode['scenes']:
            doc_key = scene['scene_id']
            general_counter = 0
            
            tokens = []
            clusters = defaultdict(list)
            subtokens_list = []
            for utterance in scene['utterances']:
                speakers = (utterance['speakers'])
                entities_list = utterance['character_entities']
    #             if len(speakers) > 1:
    #                 print(speakers)
                
                utterance_clusters = defaultdict(list)
                utterance_map = {}
                
                for idx, (sent, per_sent_entities) in enumerate(zip(utterance['tokens'], utterance['character_entities'])):
                    sent_word_idx = 0
                    sentence_token_map = {}
                    
                    for token_idx, token in enumerate(sent):
                        sentence_token_map[token_idx] = [len(subtokens_list)]
                        subtokens_list.extend(tokenizer.tokenize(token))
                        sentence_token_map[token_idx].append(len(subtokens_list))
                    
                    for entity in per_sent_entities:
                        characters  = tuple(sorted(entity[2:]))
                        token_start, token_end = entity[:2]
                        
                        span_start = sentence_token_map[token_start][0]
                        span_end = sentence_token_map[token_end - 1][1] - 1
                        utterance_clusters[characters].append((span_start, span_end))
#                         for character in characters:
#                             utterance_clusters[character] += 1
#                         uniq_entities.update(characters)
                
                for character in utterance_clusters:
#                     clusters[character] += utterance_clusters[character]
                    if character != '#GENERAL#':
                        clusters[character].extend(utterance_clusters[character])
                    else:
                        clusters[character + str(general_counter)] = utterance_clusters[character]
                        general_counter += 1
                        
                        
            
            cluster_lens.extend([len(cluster) for cluster in clusters.values()])
            
#             print(clusters)
#             print(tokenizer.convert_tokens_to_string(subtokens_list))
#             for person, ment_list in clusters.items():
#                 print(person)
#                 for ment_start, ment_end in ment_list:
#                     print(tokenizer.convert_tokens_to_string(subtokens_list[ment_start: ment_end + 1]))
                
#             break
        
#         break
#     break


print(sum(cluster_lens)/len(cluster_lens))

5.117437337942956
